In [1]:
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

In [2]:
with open('../eeg/digit/data.pkl', 'rb') as f:
	data = pickle.load(f, encoding='latin1')

In [3]:
x_eval = data['x_train'][:4000]
y_eval = data['y_train'][:4000]
x_train = data['x_train'][4000:]
y_train = data['y_train'][4000:]
x_test = data['x_test']
y_test = data['y_test']

In [4]:
class EEGDataset(torch.utils.data.Dataset):
	def __init__(self, data, labels):
		self.data = data
		self.labels = labels
	
	def __len__(self):
		return len(self.data)
	
	def __getitem__(self, idx):
		return torch.tensor(self.data[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.float32)

In [5]:
class CNNModel(nn.Module):
	def __init__(self, channels, observations, num_classes):
		super(CNNModel, self).__init__()
		self.bn1 = nn.BatchNorm2d(channels)
		self.conv1 = nn.Conv2d(1, observations, kernel_size=(1, 4))
		self.relu1 = nn.ReLU()
		self.conv2 = nn.Conv2d(observations, 25, kernel_size=(channels, 1))
		self.relu2 = nn.ReLU()
		self.maxpool1 = nn.MaxPool2d(kernel_size=(1, 3))
		self.conv3 = nn.Conv2d(1, 50, kernel_size=(4, 25))
		self.relu3 = nn.ReLU()
		self.maxpool2 = nn.MaxPool2d(kernel_size=(1, 3))
		self.conv4 = nn.Conv2d(1, 100, kernel_size=(50, 2))
		self.flatten = nn.Flatten()
		self.bn2 = nn.BatchNorm1d(100)
		self.fc1 = nn.Linear(100, 100)
		self.bn3 = nn.BatchNorm1d(100)
		self.fc2 = nn.Linear(100, num_classes)

	def forward(self, x):
		x = self.bn1(x)
		x = x.permute(0, 3, 1, 2)
		x = self.conv1(x)
		x = self.relu1(x)
		x = self.conv2(x)
		x = self.relu2(x)
		x = self.maxpool1(x)
		x = x.permute(0, 2, 3, 1)
		x = self.conv3(x)
		x = self.relu3(x)
		x = x.permute(0, 3, 1, 2)
		x = self.maxpool2(x)
		x = self.conv4(x)
		x = self.flatten(x)
		x = self.bn2(x)
		x = self.fc1(x)
		x = self.bn3(x)
		x = self.fc2(x)
		return x

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
train_loader = DataLoader(EEGDataset(x_train, y_train), batch_size=64, shuffle=True)
eval_loader = DataLoader(EEGDataset(x_eval, y_eval), batch_size=64, shuffle=False)
test_loader = DataLoader(EEGDataset(x_test, y_test), batch_size=64, shuffle=False)

In [8]:
model = CNNModel(channels=14, observations=32, num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, nesterov=True)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)

In [9]:
# Training
def train(model, loader, criterion, optimizer):
	model.train()
	running_loss = 0.0
	for i, (x, y) in enumerate(loader):
		x, y = x.to(device), y.to(device)
		optimizer.zero_grad()
		output = model(x)
		loss = criterion(output, y)
		loss.backward()
		optimizer.step()
		running_loss += loss.item()
	return running_loss / len(loader)

# Evaluation
def evaluate(model, loader, criterion):
	model.eval()
	running_loss = 0.0
	correct = 0
	total = 0
	with torch.no_grad():
		for i, (x, y) in enumerate(loader):
			x, y = x.to(device), y.to(device)
			output = model(x)
			loss = criterion(output, y)
			running_loss += loss.item()
			_, predicted = torch.max(output, 1)
			y = torch.argmax(y, 1)
			total += y.size(0)
			correct += (predicted == y).sum().item()
	return running_loss / len(loader), correct / total

In [10]:
for epoch in range(150):
	train_loss = train(model, train_loader, criterion, optimizer)
	scheduler.step()
	eval_loss, eval_acc = evaluate(model, eval_loader, criterion)
	test_loss, test_acc = evaluate(model, test_loader, criterion)
	print(f'Epoch {epoch+1:03d}: Train Loss = {train_loss:.4f}, Eval Loss = {eval_loss:.4f}, Eval Acc = {eval_acc:.4f}, Test Loss = {test_loss:.4f}, Test Acc = {test_acc:.4f}')

Epoch 001: Train Loss = 1.8724, Eval Loss = 1.6255, Eval Acc = 0.4570, Test Loss = 2.2339, Test Acc = 0.3034
Epoch 002: Train Loss = 1.4262, Eval Loss = 1.3599, Eval Acc = 0.5228, Test Loss = 2.1685, Test Acc = 0.3385
Epoch 003: Train Loss = 1.1712, Eval Loss = 1.0382, Eval Acc = 0.6595, Test Loss = 2.0536, Test Acc = 0.4319
Epoch 004: Train Loss = 1.0033, Eval Loss = 1.0759, Eval Acc = 0.6352, Test Loss = 2.1584, Test Acc = 0.4105
Epoch 005: Train Loss = 0.8888, Eval Loss = 0.8013, Eval Acc = 0.7442, Test Loss = 1.9911, Test Acc = 0.4700
Epoch 006: Train Loss = 0.7802, Eval Loss = 0.7138, Eval Acc = 0.7710, Test Loss = 1.9759, Test Acc = 0.5023
Epoch 007: Train Loss = 0.7155, Eval Loss = 0.6185, Eval Acc = 0.8257, Test Loss = 1.8968, Test Acc = 0.5392
Epoch 008: Train Loss = 0.6556, Eval Loss = 0.5378, Eval Acc = 0.8370, Test Loss = 1.8760, Test Acc = 0.5526
Epoch 009: Train Loss = 0.5997, Eval Loss = 0.5961, Eval Acc = 0.8317, Test Loss = 1.9281, Test Acc = 0.5432
Epoch 010: Train Lo

In [11]:
# Save model
torch.save(model.state_dict(), 'eeg_cnn.pth')